In [1]:
import numpy as np
import pandas as pd
import os
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import seaborn as sns
import re
from shutil import copy2
import cv2

from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import pocketsphinx

### Bag 0 : Trial n Error

In [12]:
def load_images_from_folder(folder):
    images = []
    filenames = []
    
    def extract_id(filename):
        match = re.search(r'id_(\d+)_', filename)
        return int(match.group(1)) if match else -1
    
    file_list = [f for f in os.listdir(folder) if f.endswith(".jpg") or f.endswith(".png")]
    file_list.sort(key=extract_id)
    
    for filename in file_list:
        img_path = os.path.join(folder, filename)
        img = Image.open(img_path).convert('RGB') 
        img = img.resize((128, 128))
        img_array = np.array(img)
        images.append((img_array, filename))
    
    return images

folder_path = r"C:\Users\gagah\Desktop\KKN\data\Train Images 13440x32x32\train"
images_with_filenames = load_images_from_folder(folder_path)

images = np.array([img[0] for img in images_with_filenames])
filenames = [img[1] for img in images_with_filenames]

print(f'Total data gambar: {len(images)}')
print(f'Shape setiap gambar: {images[0].shape}')

Total data gambar: 13440
Shape setiap gambar: (128, 128, 3)


In [13]:
images_with_filenames

[(array([[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         ...,
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]]], dtype=uint8),
  'id_1_label_1.png'),
 (array([[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,


In [ ]:
# Memisahkan gambar berdasarkan label
label_dict = {}
for img_array, filename in images_with_filenames:
    match = re.search(r'label_(\d+)', filename)
    if match:
        label = int(match.group(1))
        if label not in label_dict:
            label_dict[label] = []
        label_dict[label].append((img_array, filename))

# Menampilkan gambar per label
for label, images in label_dict.items():
    fig, axes = plt.subplots(1, 8, figsize=(10, 2))
    axes = axes.ravel()
    for i in range(min(8, len(images))):  # Hanya menampilkan maksimal 6 gambar per label
        axes[i].imshow(images[i][0])
        axes[i].axis('off')
        axes[i].set_title(images[i][1], fontsize=8)
    plt.suptitle(f'Label {label}')
    plt.tight_layout()
    plt.show()

In [9]:
def create_folders_by_label(source_folder, destination_folder):
    # Membuat folder tujuan jika belum ada
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    file_list = [f for f in os.listdir(source_folder) if f.endswith(".jpg") or f.endswith(".png")]
    
    for filename in file_list:
        match = re.search(r'id_(\d+)_label_(\d+)', filename)
        if match:
            img_id = match.group(1)
            label = match.group(2)
            
            # Membuat path folder label
            label_folder = os.path.join(destination_folder, f'lab_{label}')
            if not os.path.exists(label_folder):
                os.makedirs(label_folder)
            
            # Menyalin file ke folder tujuan dengan nama file baru
            src_path = os.path.join(source_folder, filename)
            dest_path = os.path.join(label_folder, f'id_{img_id}.png')
            copy2(src_path, dest_path)

source_folder_path = r"C:\Users\gagah\Desktop\KKN\data\Train Images 13440x32x32\train"
destination_folder_path = r"C:\Users\gagah\Desktop\KKN\data\Data_Train_Images"

create_folders_by_label(source_folder_path, destination_folder_path)


error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [11]:
### Bag 1 : Classification Hijaiyah

In [ ]:
TRAINING_DIR = destination_folder_path

model_dir = 'model'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=2,
    shear_range=0.4, 
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=TRAINING_DIR,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training' 
)

validation_generator = datagen.flow_from_directory(
    directory=TRAINING_DIR,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(42, activation='relu'),
    tf.keras.layers.Dense(28, activation='softmax'),
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(model_dir, 'model_{epoch:02d}_{val_accuracy:.2f}.h5'),  # Nama file dengan epoch dan akurasi validasi
    monitor='val_accuracy',    # Metrik yang dipantau
    save_best_only=True,       # Hanya menyimpan model terbaik
    mode='max',                # Mode 'max' karena kita ingin akurasi maksimum
    verbose=1                  # Menampilkan log setiap kali model disimpan
)

model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[checkpoint_callback]
)

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

def load_and_preprocess_image(img_path, target_size=(128, 128)):
    img = Image.open(img_path).convert('RGB')
    img = img.resize(target_size)
    img_array = np.array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

def load_images_from_folders(base_folder):
    images = []
    labels = []
    class_names = sorted(os.listdir(base_folder))
    
    for class_index, class_name in enumerate(class_names):
        class_folder = os.path.join(base_folder, class_name)
        if os.path.isdir(class_folder):
            for filename in os.listdir(class_folder):
                if filename.endswith(".png"):
                    img_path = os.path.join(class_folder, filename)
                    img_array = load_and_preprocess_image(img_path)
                    images.append(img_array)
                    labels.append(class_index)
    
    images = np.vstack(images)  # Combine all images into one large array
    labels = np.array(labels)
    return images, labels, class_names

# Load the model
model = load_model('model/model_38_0.86.h5')

# Path to the dataset folder
base_folder = 'data/Data_Train_Images'

# Load and preprocess images
images, true_labels, class_names = load_images_from_folders(base_folder)

# Predict the class
predictions = model.predict(images)
predicted_classes = np.argmax(predictions, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_classes)

# Mapping from original class names to new labels
label_mapping = {
    'lab_1': 'alif',
    'lab_2': 'ba',
    'lab_3': 'ta',
    'lab_4': 'tsa',
    'lab_5': 'jim',
    'lab_6': 'ha',
    'lab_7': 'kho',
    'lab_8': 'dal',
    'lab_9': 'dzal',
    'lab_10': 'ra',
    'lab_11': 'zai',
    'lab_12': 'sin',
    'lab_13': 'syin',
    'lab_14': 'shad',
    'lab_15': 'dhad',
    'lab_16': 'tha',
    'lab_17': 'dha',
    'lab_18': 'ain',
    'lab_19': 'ghain',
    'lab_20': 'fa',
    'lab_21': 'qaf',
    'lab_22': 'kaf',
    'lab_23': 'lam',
    'lab_24': 'mim',
    'lab_25': 'nun',
    'lab_26': 'ha',
    'lab_27': 'waw',
    'lab_28': 'ya'
}

# Ensure the class names are in the desired order
class_names_sorted = sorted(class_names, key=lambda x: int(x.split('_')[1]))
new_labels_sorted = [label_mapping[label] for label in class_names_sorted]

# Display confusion matrix with new labels
plt.figure(figsize=(12, 10))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=new_labels_sorted, yticklabels=new_labels_sorted)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Print classification report with new labels
print(classification_report(true_labels, predicted_classes, target_names=new_labels_sorted))


### Bag 1 : Classification Hijaiyah

In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Load the model
model = load_model('model/model_38_0.86.h5')

# Mapping from original class names to new labels
label_mapping = {
    0: 'alif',
    1: 'ba',
    2: 'ta',
    3: 'tsa',
    4: 'jim',
    5: 'ha',
    6: 'kho',
    7: 'dal',
    8: 'dzal',
    9: 'ra',
    10: 'zai',
    11: 'sin',
    12: 'syin',
    13: 'shad',
    14: 'dhad',
    15: 'tha',
    16: 'dha',
    17: 'ain',
    18: 'ghain',
    19: 'fa',
    20: 'qaf',
    21: 'kaf',
    22: 'lam',
    23: 'mim',
    24: 'nun',
    25: 'ha',
    26: 'waw',
    27: 'ya'
}



# Path to the image file you want to predict
image_path = 'data\Data_Train_Images\lab_1\id_1.png'

# Preprocess the image and get original image and numpy array
original_image, preprocessed_image = preprocess_image(image_path)

# Predict the class
prediction = model.predict(preprocessed_image)
predicted_class = np.argmax(prediction, axis=1)[0]

# Map the predicted class to the new label
predicted_label = label_mapping[predicted_class]

# Display the original image with prediction using Matplotlib
plt.figure(figsize=(6, 6))
plt.imshow(original_image)
plt.axis('off')
plt.title(f'Predicted: {predicted_label}', fontsize=15)
plt.show()

# Print original file name
print(f'Original File Name: {image_path}')
# Print predicted label
print(f'Predicted Label: {predicted_label}')

### Bag 2 : Recognize Doa Speech